In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec
import re
import nltk

In [2]:
df = pd.read_json(path_or_buf='gazeta_train.jsonl', lines=True)

In [3]:
df.head(4)

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50


In [4]:
def preprocess_txt(line):
    morpher = MorphAnalyzer()
    sw = set(get_stop_words('ru'))
    exclude = set(string.punctuation)
  
    spls = ''.join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != '']
    
    return spls

In [5]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
            
    if norm > 0:
        vector = vector / norm
        
    answers = index.get_nns_by_vector(vector, 3, )
    
    return [index_map[i] for i in answers]

In [6]:
def lower_words(text):
    lower_text = text.lower()

    return lower_text

In [7]:
sentences = [row.split() for row in df['text']]

In [8]:
w2v_model = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [9]:
def get_sentence_vector(sentence, model):
    counter = 0
    vec = np.zeros(model.vector_size)
    for word in sentence:
        if word in model.wv:
            vec += model.wv[word]
            counter += 1
    return vec / counter if counter else vec

In [10]:
w2v_index = annoy.AnnoyIndex(w2v_model.vector_size, 'angular')

for i, sentence in enumerate(df['text']):
    w2v_vec = get_sentence_vector(sentence, w2v_model)
    w2v_index.add_item(i, w2v_vec)

w2v_index.build(20)

True

In [11]:
def get_similar_tweets(tweet, model, k=5):
    tweet = preprocess_txt(tweet)
    tweet_vec = get_sentence_vector(tweet, model)
    idxs = w2v_index.get_nns_by_vector(tweet_vec, k)
    
    return df['text'].iloc[idxs].values

In [12]:
texts = ['На улицах Москвы снова праздник',
         'В Санкт-Петербурге выпал первый снег',
         'В Краснодаре нашли злоумышленников и приговорили к 5 годам строгого режима',
         'В Саратовской области ожидается большое количество осадков',
         'Блогеры против телевизионных журналистов - у кого нынче рейтинги выше?'
        ]

In [13]:
print('*' * 50)

for i, text in enumerate(texts):
    print(f'{i + 1} {text}')
    print('*' * 50)
    print(get_similar_tweets(text, w2v_model, k=5))
    print('*' * 50, end='\n\n')

**************************************************
1 На улицах Москвы снова праздник
**************************************************
["В Москве, вслед за Нью-Йорком, Лондоном, Миланом и Парижем, проходит Неделя моды сезона «Осень-зима 2016-2017» Mercedes-Benz Fashion Week Russia. С 11 по 16 марта в показах примут участие дизайнеры из России, Грузии, Казахстана, Белоруссии и Украины. На показах SLAVA ZAITSEV, Alena Akhmadullina, VIVA VOX, YASYA MINOCHKINA (Украина), Юлии Николаевой, Dasha Gauser, Julia Dalakian, PIROSMANI BY JENYA MALYGINA, IGOR GULYAEV, Goga Nikabadze (Грузия), DIMANEU - непременный ажиотаж, отдельные программы посвящены казахским и белорусским модельерам, а среди дебютантов - бренды с многообещающими названиями VIPERS, KSENIASERAYA, SORRY, I'M NOT. Лаконизм, минимализм, женственность и эпатаж на подиуме, красные губы, кроссовки и туфли на шпильках в зале: интересно, чем удивит 32-я Неделя моды. 11.03, пятница 15:00 SLAVA ZAITSEV представляет Лабораторию моды/Slava 

['Современные смартфоны, вне зависимости от их мощности, операционной системы и прочих технических параметров имеют один общий недостаток: небольшое время работы от аккумулятора. Если у планшетов время работы соотносимо с продолжительностью рабочего дня, то их братья меньшие, смартфоны, при активном использовании не протянут день без подзарядки. Да, как правило, в течение дня у каждого из нас есть возможность подзарядиться от сети, от прикуривателя автомобиля или от компьютера. Но бывает и так, что источников питания поблизости нет, а заряд заканчивается. Для таких случаев и предназначаются мобильные зарядные устройства. Зарядные устройства в данном обзоре расположены в порядке возрастания их емкости. Характеристики Масса: 52 грамма Габариты: 49х102х11,5 мм Емкость: 1200 мАч Цена: 1150 руб. Характеристики Масса: 94 г Габариты: 113x56x11 мм Емкость: 1500mAh Цена: около 1700 руб. Характеристики Масса: 88 гр. Габариты: 126x62x16 мм Емкость: 2000 мАч Цена: 2800 руб. Характеристики Масса: 8

['КХЛ не упрекнешь в непоследовательности. Как наметила курс на открытость бюджетов, так с него и не сходит. Открылись данные о прошлогодних зарплатных ведомостях клубов, о деньгах, которые энхаэловцы получили в локаутный сезон. Следующим шагом этого парада откровенности стало «бюджетное превью» — информация о ведомостях клубов, только вступающих в турнирную борьбу. 1. СКА – 1 271 188 649 2. «Ак Барс» – 1 085 187 291 3. «Металлург» Мг – 1 033 995 000 4. «Салават Юлаев» – 1 011 274 146 5. ЦСКА – 965 412 000 6. «Трактор» – 878 689 000 7. «Динамо» – 865 558 392 8. «Авангард» – 862 018 970 9. «Лев» – 741 604 520 10. «Локомотив» – 716 379 516 11. «Донбасс» – 715 767 472 12. «Атлант» – 613 105 000 13. «Барыс» – 562 403 632 14. «Нефтехимик» – 459 812 118 15. «Северсталь» – 445 590 000 16. «Сибирь» – 406 632 000 17. «Витязь» – 392 097 328 18. «Амур» – 368 326 782 19. «Автомобилист» – 351 838 000 20. «Торпедо» – 351 770 311 21. «Югра» – 347 820 000 22. «Спартак» – 324 440 000 23. «Динамо» Минск

['Банк развития Белоруссии проведет с 10 апреля встречи с инвесторами, посвященные возможному размещению гособлигаций, сообщил РИА «Новости» источник в банковских кругах. По его словам, road show пройдет в Европе и США. В зависимости от рыночных условий, банк может разместить пятилетние бонды в долларах, а также рассмотреть возможность выпуска трехлетних облигаций в белорусских рублях, выплаты по которым будут осуществляться в долларах. Организаторами выступят Citi и Raiffeisen. Банк развития Белоруссии — государственный институт национального развития республики, создан в соответствии с указом президента в июне 2011 года. Учредителями банка выступили Совет министров и Национальный банк РБ. Объем размещения госбумаг на внешних рынках президент Белоруссии Александр Лукашенко согласовал своим указом №458 «О привлечении внешних государственных займов», подписанным в конце прошлого года. Займы планируется проводить в 2018-2020 годах на внешних рынках гособлигаций на сумму, эквивалентную $2

['iPhone 5 Nokia Lumia 920 Galaxy Note 2 Amazon Kindle Paperwhite Barnes & Noble Nook HD+ «Лучшее для Android» «Компьютер» «Фото»'
 '21 мая в престижном лондонском районе Челси открывается «Шоу цветов». Перед открытием выставку посетили герцогиня Кембриджская Кэтрин и Елизавета II — они семьи осмотрели сад, в создании которого принимала участие сама Кейт Миддлтон. Источники отмечают, что проект сада под названием «Назад к природе» разрабатывали несколько месяцев, и отчасти он посвящен душевному здоровью и пользе отдыха на природе. В саду представлены развлечения для различных возрастных групп — тарзанка, домик на дереве и ручей с небольшими водопадами. Небольшую постройку, куда можно забраться детям и взрослым, разместили на ветвях каштанового дерева, а в менее интерактивных зонах разместили зеленую стену. The Duke and Duchess of Cambridge are very pleased to share some new photographs of their family as they visited the RHS Back to Nature Garden at the #ChelseaFlowerShow on Sunday aft